# Heat Equation with Variable Conductivity

![](heat_equation.png)

## Introduction

Here we solve the equation on a unit square, with a uniform internal source, with the diffusion coefficient $k(x)$ given as a generic function.
The strong form of the (linear) heat equation is given by

\begin{equation}
 -\nabla \cdot (k \nabla u) = f  \quad x \in \Omega,
\end{equation}

where $u$ is the unknown temperature field, $k$ the heat conductivity,
$f$ the heat source and $\Omega$ the domain. We will consider homogeneous Dirichlet boundary conditions such that
\begin{equation}
u(x) = 0 \quad x \in \partial \Omega,
\end{equation}
where $\partial \Omega$ denotes the boundary of $\Omega$.

The resulting weak form is given by
\begin{equation}
\int_{\Omega} k \nabla v \cdot \nabla u \ d\Omega = \int_{\Omega} f v \ d\Omega,
\end{equation}
where $v$ is a suitable test function.

## Commented Program

Now we solve the problem in JuAFEM. What follows is a program spliced with comments.

First we load the JuAFEM package.

In [2]:
using JuAFEM
#using MATLAB
using BenchmarkTools

In [3]:
include("../Utils/mesh_utils.jl")
include("../Utils/geometry_utils.jl")

**Option #1**: Generate a simple grid with 20x20 triangular elements
using `generate_grid`. The generator defaults to the unit square,
so we don't need to specify the corners of the domain.

In [4]:
const n = 20
grid = generate_grid(Triangle, (n, n));

**Option #2**: External call to `MATLAB` to generate non-uniform grid with circles inside and convert to a `JuAFEM` grid.

In [ ]:
#bbox    = [-20.0 -20.0; 20.0 20.0]/20;
#centers = [  5.5   5.5; -7.0 -0.8; -7.0 -0.8]/20;
#radii   = [        2.5;       4.9;       2.0]/20;

bbox    = [-1.0 -1.0; 1.0 1.0];
centers = [ 0.0  0.0];
radii   = [      0.5];

Nmin = 100; # points for smallest circle
h0 = 2pi*minimum(radii)/Nmin; # approximate scale
eta = 5.0; # approx ratio between largest/smallest edges

In [ ]:
# Note: add minimum angle threshold
nargout = 2
p, t = mxcall(:squareMeshWithCircles, nargout, bbox, centers, radii, h0, eta);

cells = [Triangle((t[i,1], t[i,2], t[i,3])) for i in 1:size(t,1)];
nodes = [Node((p[i,1], p[i,2])) for i in 1:size(p,1)];
grid = Grid(cells, nodes);

In [ ]:
# Manually add boundary sets for the four square edges
addfaceset!(grid, "left",   x -> x[1] ≈ bbox[1,1], all=true)
addfaceset!(grid, "right",  x -> x[1] ≈ bbox[2,1], all=true)
addfaceset!(grid, "top",    x -> x[2] ≈ bbox[1,2], all=true)
addfaceset!(grid, "bottom", x -> x[2] ≈ bbox[2,2], all=true)

### Heat conductivity $k(x)$ and heat force $f(x)$ functions
Define functions $k(x)$ and $f(x)$ for use in assembling the linear system.

In [5]:
#conductivity(x::Vec{2,T}) where T = ifelse(x⋅x > T(0.25), zero(T), one(T))
#source(x::Vec{2,T}) where T = ifelse(x⋅x > T(0.25^2), zero(T), T(5.0))
conductivity(x::Vec{2,T}) where T = one(T)
source(x::Vec{2,T}) where T = one(T)

source (generic function with 1 method)

### Trial and test functions
A `CellValues` facilitates the process of evaluating values and gradients of
test and trial functions (among other things). Since the problem
is a scalar problem we will use a `CellScalarValues` object. To define
this we need to specify an interpolation space for the shape functions.
We use Lagrange functions (both for interpolating the function and the geometry)
based on the reference "cube". We also define a quadrature rule based on the
same reference cube. We combine the interpolation and the quadrature rule
to a `CellScalarValues` object.

In [62]:
const dim = 2
ip = Lagrange{dim, RefTetrahedron, 1}()
qr = QuadratureRule{dim, RefTetrahedron}(2)
qr_face = QuadratureRule{dim-1, RefTetrahedron}(2)
cellvalues = CellScalarValues(qr, ip);
facevalues = FaceScalarValues(qr_face, ip);

### Degrees of freedom
Next we need to define a `DofHandler`, which will take care of numbering
and distribution of degrees of freedom for our approximated fields.
We create the `DofHandler` and then add a single field called `u`.
Lastly we `close!` the `DofHandler`, it is now that the dofs are distributed
for all the elements.

In [63]:
dh = DofHandler(grid)
push!(dh, :u, 1)
close!(dh);

Now that we have distributed all our dofs we can create our tangent matrix,
using `create_sparsity_pattern`. This function returns a sparse matrix
with the correct elements stored.

In [64]:
K = create_sparsity_pattern(dh);

We can inspect the pattern using the `spy` function from `UnicodePlots.jl`.
By default the stored values are set to $0$, so we first need to
fill the stored values, e.g. `K.nzval` with something meaningful.

In [65]:
#using UnicodePlots
#fill!(K.nzval, 1.0)
#spy(K; height = 25)

### Boundary conditions
In JuAFEM constraints like Dirichlet boundary conditions
are handled by a `ConstraintHandler`.

In [66]:
ch = ConstraintHandler(dh);

Next we need to add constraints to `ch`. For this problem we define
homogeneous Dirichlet boundary conditions on the whole boundary, i.e.
the `union` of all the face sets on the boundary.

In [67]:
∂Ω = union(getfaceset.(grid, ["left", "right", "top", "bottom"])...);

Now we are set up to define our constraint. We specify which field
the condition is for, and our combined face set `∂Ω`. The last
argument is a function which takes the spatial coordinate $x$ and
the current time $t$ and returns the prescribed value. In this case
it is trivial -- no matter what $x$ and $t$ we return $0$. When we have
specified our constraint we `add!` it to `ch`.

In [68]:
dbc = Dirichlet(:u, ∂Ω, (x, t) -> 0)
add!(ch, dbc);

We also need to `close!` and `update!` our boundary conditions. When we call `close!`
the dofs which will be constrained by the boundary conditions are calculated and stored
in our `ch` object. Since the boundary conditions are, in this case,
independent of time we can `update!` them directly with e.g. $t = 0$.

In [69]:
close!(ch)
update!(ch, 0.0);

### Assembling the linear system
Now we have all the pieces needed to assemble the linear system, $K u = f$.
We define a function, `doassemble` to do the assembly, which takes our `cellvalues`,
the sparse matrix and our DofHandler as input arguments. The function returns the
assembled stiffness matrix, and the force vector.

In [165]:
function doassemble(cellvalues::CellScalarValues{dim},
                    facevalues::FaceScalarValues{dim},
                    K::SparseMatrixCSC,
                    dh::DofHandler;
                    evalfaceintegrals=false) where {dim}
    # We allocate the element stiffness matrix and element force vector
    # just once before looping over all the cells instead of allocating
    # them every time in the loop.
    n_basefuncs = getnbasefunctions(cellvalues)
    Ke = zeros(n_basefuncs, n_basefuncs)
    fe = zeros(n_basefuncs)
    
    # Next we define the global force vector `f` and use that and
    # the stiffness matrix `K` and create an assembler. The assembler
    # is just a thin wrapper around `f` and `K` and some extra storage
    # to make the assembling faster.
    f = zeros(ndofs(dh))
    assembler = start_assemble(K, f)
    
    # It is now time to loop over all the cells in our grid. We do this by iterating
    # over a `CellIterator`. The iterator caches some useful things for us, for example
    # the nodal coordinates for the cell, and the local degrees of freedom.
    @inbounds for cell in CellIterator(dh)
        # Always remember to reset the element stiffness matrix and
        # force vector since we reuse them for all elements.
        fill!(Ke, 0)
        fill!(fe, 0)
        
        # Get the coordinates of the cell
        coords = getcoordinates(cell)
        
        # For each cell we also need to reinitialize the cached values in `cellvalues`.
        reinit!(cellvalues, cell)
        
        # It is now time to loop over all the quadrature points in the cell and
        # assemble the contribution to `Ke` and `fe`. The integration weight
        # can be queried from `cellvalues` by `getdetJdV`, and the quadrature
        # coordinate can be queried from `cellvalues` by `spatial_coordinate`
        for q_point in 1:getnquadpoints(cellvalues)
            dΩ = getdetJdV(cellvalues, q_point)
            coords_qp = spatial_coordinate(cellvalues, q_point, coords)
            
            # calculate the heat conductivity and heat source at point `coords_qp`
            k_cond = conductivity(coords_qp)
            f_source = source(coords_qp)
            
            # For each quadrature point we loop over all the (local) shape functions.
            # We need the value and gradient of the testfunction `v` and also the gradient
            # of the trial function `u`. We get all of these from `cellvalues`.
            for i in 1:n_basefuncs
                v  = shape_value(cellvalues, q_point, i)
                ∇v = shape_gradient(cellvalues, q_point, i)
                fe[i] += f_source * v * dΩ
                for j in 1:n_basefuncs
                    ∇u = shape_gradient(cellvalues, q_point, j)
                    Ke[i, j] += k_cond * (∇v ⋅ ∇u) * dΩ
                end
            end
        end
        
        # Now, loop over the edges of the cell for contributions to `Ke`
        #   If "Neumann Boundary" is a subset of boundary points, use:
        #     `onboundary(cell, face) && (cellid(cell), face) ∈ getfaceset(grid, "Neumann Boundary")`
        if evalfaceintegrals
            for face in 1:nfaces(cell)
                #if !onboundary(cell, face)
                    # Initialize face values
                    reinit!(facevalues, cell, face)

                    for q_point in 1:getnquadpoints(facevalues)
                        dΓ = getdetJdV(facevalues, q_point)
                        coords_qp = spatial_coordinate(facevalues, q_point, coords)

                        # calculate the heat conductivity and heat source at point `coords_qp`
                        k_cond = conductivity(coords_qp)
                        kdΓ = k_cond * dΓ

                        for i in 1:getnbasefunctions(facevalues)
                            n = getnormal(facevalues, q_point)
                            v = shape_value(facevalues, q_point, i)
                            vkdΓ = v * kdΓ
                            for j in 1:n_basefuncs
                                ∇u = shape_gradient(facevalues, q_point, j)
                                Ke[i,j] += (∇u⋅n) * vkdΓ
                            end
                        end
                    end
                #end
            end
        end
        
        # The last step in the element loop is to assemble `Ke` and `fe`
        # into the global `K` and `f` with `assemble!`.
        assemble!(assembler, celldofs(cell), fe, Ke)
    end
    return K, f
end

doassemble (generic function with 1 method)

### Solution of the system
The last step is to solve the system. First we call `doassemble`
to obtain the global stiffness matrix `K` and force vector `f`.
Then, to account for the boundary conditions, we use the `apply!` function.
This modifies elements in `K` and `f` respectively, such that
we can get the correct solution vector `u` by using `\`.

In [166]:
fill!(K,0.0)
K, f = doassemble(cellvalues, facevalues, K, dh; evalfaceintegrals=false);
apply!(K, f, ch)
u = K \ f;

In [167]:
@show (minimum(u), maximum(u), norm(u));

(minimum(u), maximum(u), norm(u)) = (0.0, 0.29410683693356127, 3.2940423035182347)


Solve the system including the boundary integrals

In [168]:
Kb = copy(K); fill!(Kb,0.0)
Kb, fb = doassemble(cellvalues, facevalues, Kb, dh; evalfaceintegrals=true);
apply!(Kb, fb, ch)
ub = Kb \ fb;

In [169]:
@show (minimum(ub), maximum(ub), norm(ub));

(minimum(ub), maximum(ub), norm(ub)) = (0.0, 0.14705341846678055, 1.6470211517591142)


In [170]:
u ≈ 2ub

true

In [171]:
2K ≈ Kb

true

### Exporting to VTK
To visualize the result we export the grid and our field `u`
to a VTK-file, which can be viewed in e.g. [ParaView](https://www.paraview.org/).

In [ ]:
vtk_grid("heat_equation_variable_conductivity", dh) do vtk
    vtk_point_data(vtk, dh, u)
end

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*